# Lời giải cơ bản cho cuộc thi

Đây là hướng dẫn một phương pháp cơ bản dùng để phát hiện hướng di chuyển (MOI) của phương tiện giao thông trong vùng trong vùng quan sát (ROI) cùng thời điểm phương tiện rời khỏi ROI.

Hướng dẫn này sử dụng các mã nguồn mở từ các nguồn sau:

* [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)

* [Simple online and realtime tracking](https://github.com/abewley/sort)



---


Tổng quan phương pháp cơ bản được trình bày như sau:
* **Phát hiện phương tiện giao thông trong từng frame của video**: Sử dụng TensorFlow Object Detection API. Kết quả trả về là một danh sách các bounding box ứng với tất cả các phương tiện giao thông trong ảnh.
* **Theo vết (multiple objects tracking)**: Dựa vào IOU (chỉ số đo đạc mức độ trùng lắp của hai bounding box), các bounding box ở các frame liên tiếp sẽ được gom nhóm và từ đó sẽ hình thành quỹ đạo di chuyển của chính phương tiện đó.
* **Xác định hướng di chuyển (MOI) dựa trên quỹ đạo**: MOI của phương tiện sẽ được lựa chọn dựa trên độ tương đồng (ở đây sử dụng Cosine Similarity Score) giữa quỹ đạo của mỗi phương tiện và các MOI sẽ được tính toán.   

**Lưu ý**: Trong lời giải cơ bản này, chỉ minh họa việc phát hiện và đếm phương tiện xe máy. Bạn cần chỉnh sửa và cải tiến để có thể đếm được tất cả phương tiện giao thông mà đề bài yêu cầu.


## Imports and Setup

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 27kB/s 
     |████████████████████████████████| 3.0MB 38.6MB/s 
     |████████████████████████████████| 460kB 43.5MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


# Cấu hình thư mục lưu trữ dữ liệu

Hướng dẫn này được chạy trên [Google Colab](https://colab.research.google.com/) (xem phần [Colab FAQ](https://research.google.com/colaboratory/faq.html) để biết thêm thông tin cần lưu ý)

Đoạn chương trình sau cấu hình đường dẫn thư mục Google Drive của bạn để thuận tiện cho việc chạy thí nghiệm nhiều lần và lưu trữ dữ liệu.

In [ ]:
https://drive.google.com/drive/folders/1olncYMTJocpEXh3l12VVbHLDjipAtKPo?usp=sharing

In [ ]:

# Mount "My Drive" into /content/drive
from google.colab import drive

google_drive_dir = 'VietAI'  # @param

drive.mount('/content/drive')

mount_point = '/content/drive/My Drive/{}'.format(google_drive_dir)

# Change the root directory to your mount_point
#% cd '$mount_point'

Mounted at /content/drive


In [ ]:
% cd '/content'

/content


In [ ]:
import os
import pathlib

# Clone the baseline repository 
if not pathlib.Path('ai-challenge-baseline').exists():
  ! git clone https://github.com/hcmcaic/ai-challenge-baseline

# 
#% cd ai-challenge-baseline


Cloning into 'ai-challenge-baseline'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 40 (delta 9), reused 26 (delta 7), pack-reused 11
Unpacking objects: 100% (40/40), done.


In [ ]:
%cd /content

/content


In [ ]:
!rm -rf '/content/ai-challenge-baseline/models'

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2301, done.
remote: Counting objects: 100% (2301/2301), done.
remote: Compressing objects: 100% (1993/1993), done.
remote: Total 2301 (delta 564), reused 940 (delta 285), pack-reused 0
Receiving objects: 100% (2301/2301), 30.57 MiB | 5.59 MiB/s, done.
Resolving deltas: 100% (564/564), done.


In [ ]:
% cd ai-challenge-baseline

/content/ai-challenge-baseline


In [ ]:
%cd /content/models/research/

/content/models/research


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1598322 sha256=638be94434cb6c37c8aeda6949d7e6d3f8ccc4920c7740d04cee6f05253f1ae9
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xx2f0_n/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=889e0f857a4e49a80c3f12c5bf4b6ad09068db858b3a42eb479db6ca9638dd3b
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=f0479a452f473632bff425417b207a5e6d6834022ff629bf95274f4ad92316de
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=5dbe9e102abec43951b4f1754dcc8e9d036207904a45d224b34a

bash: line 1: cd: models/research/: No such file or directory
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.25.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [ ]:
# Install the requirements package for SORT source code
! pip install filterpy scikit-image lap

     |████████████████████████████████| 184kB 2.6MB/s 
     |████████████████████████████████| 1.5MB 8.2MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110450 sha256=45fd1b00604d1cbb622720148eeef0f76dd11d61c66c9cbb568d07004353b52f
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
  Created wheel for lap: filename=lap-0.4.0-cp36-cp36m-linux_x86_64.whl size=1589030 sha256=fe7413d880bf79590b6031aeac746aa1baed0ecd249b28018b25aed4ab5df07f
  Stored in directory: /root/.cache/pip/wheels/da/3e/af/eddcd6ffaa27df8d0ddac573758f8953c4e57c64c4c8c8b7d0
Successfully built filterpy lap


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from time import time
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
import csv
import tensorflow as tf
import os
import pathlib



In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

# Đọc dữ liệu từ video

Đoạn chương trình dưới đây dùng để trích xuất tất cả các khung ảnh (frame) của video và lưu lại dưới dạng các ảnh riêng lẻ trong thư mục tạm.

In [ ]:

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list


# Chuẩn bị model


Đoạn chương trình sau sử dụng Detection API có sẵn, bạn có thể lựa chọn hoặc thay đổi các detection architecture cùng với backbone mà model zoo cung cấp

In [ ]:
print(model_config_file)
print(model_weight_file)

Tải và giải nén pretrained model

In [ ]:
%cd /content

/content


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.builders import model_builder


class Detector(object):
    def __init__(self, path_config, path_ckpt, path_to_labels):
        self.path_config = path_config
        self.path_ckpt = path_ckpt
        self.label_path = path_to_labels

        self.category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)
        self.detection_model = self.load_model()
        self.detection_scores = None
        self.detection_boxes = None
        self.detection_classes = None

    def detect_fn(self, image):
        """Detect objects in image."""
        image, shapes = self.detection_model.preprocess(image)
        prediction_dict = self.detection_model.predict(image, shapes)
        detections = self.detection_model.postprocess(prediction_dict, shapes)
        return detections

    def load_model(self):
        # Load pipeline config and build a detection model
        configs = config_util.get_configs_from_pipeline_file(self.path_config)
        model_config = configs['model']
        detection_model = model_builder.build(model_config=model_config, is_training=False)

        # Restore checkpoint
        ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
        ckpt.restore(self.path_ckpt).expect_partial()

        return detection_model

    def predict(self, image):
        original_img = np.copy(image)

        image = np.asarray(image)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image, 0), dtype=tf.float32)
        detections = self.detect_fn(input_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(detections.pop('num_detections'))
        # num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        self.detection_scores = detections['detection_scores']
        self.detection_classes = detections['detection_classes']
        self.detection_boxes = detections['detection_boxes']

        # draw bounding boxes and labels
        image, coordinate_dict = self.draw(image)

        return image, original_img, coordinate_dict

    def draw(self, img):
        coordinate_dict = dict()
        height, width, _ = img.shape
        li = []

        for i, score in enumerate(self.detection_scores):
            self.detection_classes[i] += 1
            # if background, ignore
            # if self.detection_classes[i] == 0:
            #     continue
            # if self.detection_classes[i] != 3 and self.detection_classes[i] != 4 and self.detection_classes[i] != 6 and self.detection_classes[i] != 8:
            #     continue
            # If class is motorcycle 
            if self.detection_classes[i] == 1:
                if score < 0.3:
                  continue
            # If class is car
            if self.detection_classes[i] == 2:
                if score < 0.5:
                  continue
            # If class is bus
            if self.detection_classes[i] == 3:
                if score < 0.7:
                  continue
            # If class is truck , container
            if self.detection_classes[i] == 4:
                if score < 0.7:
                  continue      
            print("Index {} at score {} and class {} ".format(i , score, self.detection_classes[i]))                        
            label = "index " + str(i) + " class " + str(self.category_index[self.detection_classes[i]]['name']) 
            ymin, xmin, ymax, xmax = self.detection_boxes[i]
            real_xmin, real_ymin, real_xmax, real_ymax = int(xmin * width), int(ymin * height), int(xmax * width), int(
                ymax * height)

            #curr = real_xmax * real_ymax - real_ymin * real_xmin
            curr_bb = {"xmin" : real_xmin , "xmax" : real_xmax , "ymin" : real_ymin , "ymax" :real_ymax }
            status = is_overlap(curr_bb, li)
            if status == 1:
                continue
            print("Is not ovelaped")
            li.append(curr_bb)
            # check overlap bounding boxes

            cv2.rectangle(img, (real_xmin, real_ymin), (real_xmax, real_ymax), (0, 255, 0), 2)
            cv2.putText(img, label, (real_xmin, real_ymin), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
            coordinate_dict[label] = (real_xmin, real_ymin, real_xmax, real_ymax)

        return img, coordinate_dict

def get_IOU(bb1 , bb2):
    x_left = max(bb1['xmin'], bb2['xmin'])
    y_top = max(bb1['ymin'], bb2['ymin'])
    x_right = min(bb1['xmax'], bb2['xmax'])
    y_bottom = min(bb1['ymax'], bb2['ymax'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['xmax'] - bb1['xmin']) * (bb1['ymax'] - bb1['ymin'])
    bb2_area = (bb2['xmax'] - bb2['xmin']) * (bb2['ymax'] - bb2['ymin'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0
    assert iou <= 1
    return iou

def is_overlap(bb , list_bounding_box):
    for bounding_box in list_bounding_box:
        if (get_IOU(bb , bounding_box) > 0.5):
          return 1
    return 0

def check_overlap(curr, li):
    for va in li:
        # overlap
        if abs(va - curr) < 1000:
            return 1
    return 0

# Lấy thông tin MOI và ROI từ json
Đọc file json ứng với mỗi video chứa thông tin ROI và MOI

In [ ]:
import json
def load_zone_anno(json_filename):
  """
  Load the json with ROI and MOI annotation.

  """
  with open(json_filename) as jsonfile:
    dd = json.load(jsonfile)
    polygon = [(int(x), int(y)) for x, y in dd['shapes'][0]['points']]
    paths = {}
    for it in dd['shapes'][1:]:
      kk = str(int(it['label'][-2:]))
      paths[kk] = [(int(x), int(y)) for x, y
              in it['points']]
  return polygon, paths
  

In [ ]:
polygon, paths = load_zone_anno('/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/zones_movement_paths/cam_01.json')


In [ ]:
polygon

[(1, 341), (150, 210), (963, 249), (1278, 464), (0, 462)]

In [ ]:
paths

{'1': [(347, 210), (196, 548)], '2': [(850, 512), (736, 220)]}

# Phát hiện hướng duy chuyển của các xe máy

Trong hướng dẫn này chỉ đếm một loại phương tiện là xe máy. Đối với source code hiện tại, mỗi loại phương tiện cần tạo một tracker khác nhau.

Hàm kiểm tra phương tiện phát hiện được có nằm trong ROI

In [ ]:
%cd '/content/ai-challenge-baseline'

/content/ai-challenge-baseline


In [ ]:

from solution_baseline import bb_polygon
def check_bbox_intersect_polygon(polygon, bbox):
  """
  
  Args:
    polygon: List of points (x,y)
    bbox: A tuple (xmin, ymin, xmax, ymax)
  
  Returns:
    True if the bbox intersect the polygon
  """
  x1, y1, x2, y2 = bbox
  bb = [(x1,y1), (x2, y1), (x2,y2), (x1,y2)]
  return bb_polygon.is_bounding_box_intersect(bb, polygon)


In [ ]:

from solution_baseline.sort import *



In [ ]:
image_dir = '/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01'

In [ ]:
detector = Detector('/content/drive/My Drive/VietAI/training1/pipeline1.config'
                    ,'/content/drive/My Drive/VietAI/training1/ckpt-12'
                    ,'/content/drive/My Drive/VietAI/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt')

In [ ]:
import time
from google.colab.patches import cv2_imshow

In [ ]:
# Create an motobikes tracker with default parameter.
# Please read the sort documentation for the custom paramenters.
from IPython.display import clear_output
from tqdm import tqdm
import os

bus_tracker = Sort()
car_tracker = Sort()
moto_tracker = Sort()

# If you want to track another vehicle class, you need to declare a new tracker.
# truck_tracker = Sort()

track_dict_bus = {}
track_dict_car = {}
track_dict_moto = {}

N_FRAMES = 6010

print("running")
for frame_id in tqdm(range(0, N_FRAMES ,1)):
  if frame_id % 10 == 0:
    clear_output(wait=True)
  image_path = os.path.join(image_dir, '{}.jpg'.format(frame_id))
  # print("Image path :")
  print(image_path)
  if not os.path.isfile(image_path):
    continue
  image_np = load_image_into_numpy_array(image_path)

  # track_image_for_bus = image_np.copy()
  # track_image_for_car = image_np.copy()
  # track_image_for_moto = image_np.copy()

  im_width, im_height, _ = image_np.shape
  input_tensor = tf.convert_to_tensor(
      np.expand_dims(image_np, 0), dtype=tf.float32)
  #detections, predictions_dict, shapes = detect_fn(input_tensor)
  #image, original_image, coordinate_dict = detector.predict(image_np)
  #cv2_imshow(image)

  # print("Visualize after get tracker")
  detections = detector.detect_fn(input_tensor)

  boxes = detections['detection_boxes'][0]
  scores = detections['detection_scores'][0]
  classes = detections['detection_classes'][0]

  dets_bus = []
  dets_car = []
  dets_moto = []
  #count = -1
  for bb, s, c in zip(boxes, scores, classes):
    num_class = c + 1
    #count = count + 1
    if num_class == 1:
      if s < 0.3:
        continue  
    if num_class == 2:
      if s < 0.5:
        continue  
    if num_class == 3:
      if s < 0.7:
        continue  
    if num_class == 4:
      if s < 0.7:
        continue  
    ymin, xmin, ymax, xmax = bb.numpy()
    xmin, ymin, xmax, ymax = int(xmin*im_height), int(ymin*im_width), int(xmax*im_height), int(ymax*im_width)
    if check_bbox_intersect_polygon(polygon, (xmin, ymin, xmax, ymax)):
      #print("index {} class {} Is intersect  ROI".format(count , num_class))
      if num_class == 3:
      # check if the bbox is in ROI
        dets_bus.append([xmin, ymin, xmax, ymax, s.numpy()])
      if num_class == 2:
        dets_car.append([xmin, ymin, xmax, ymax, s.numpy()])
      if num_class == 1:
        dets_moto.append([xmin, ymin, xmax, ymax, s.numpy()])
    #else :
      #print("index {} class {} Is not intersect of ROI".format(count , num_class))



  dets_bus = np.array(dets_bus)
  dets_car = np.array(dets_car)
  dets_moto = np.array(dets_moto)

  if dets_bus.size != 0:
    trackers_bus = bus_tracker.update(dets_bus)
  # print('trackers')
  # print(trackers)

    for xmin, ymin, xmax, ymax, track_id in trackers_bus:
      track_id = int(track_id)
      #  Visualize tracker
      #cv2.rectangle(track_image_for_bus, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
      #cv2.putText(track_image_for_bus, "Bus Track " + str(track_id), (int(xmax), int(ymax)), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
      ##print(track_id)
      if track_id not in track_dict_bus.keys():
        track_dict_bus[track_id] = [(xmin, ymin, xmax, ymax, frame_id)]
      else:
        track_dict_bus[track_id].append((xmin, ymin, xmax, ymax, frame_id))
    # print("Trackdict")
    # print(track_dict)
    print("Track number in bus: {}".format(len(track_dict_bus.keys())))
    #cv2_imshow(track_image_for_bus) 

  if dets_car.size != 0:
    trackers_car = car_tracker.update(dets_car)
  # print('trackers')
  # print(trackers)

    for xmin, ymin, xmax, ymax, track_id in trackers_car:
      track_id = int(track_id)
      #  Visualize tracker
      #cv2.rectangle(track_image_for_car, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
      #cv2.putText(track_image_for_car, "Car Track " + str(track_id), (int(xmax), int(ymax)), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
      ##print(track_id)
      if track_id not in track_dict_car.keys():
        track_dict_car[track_id] = [(xmin, ymin, xmax, ymax, frame_id)]
      else:
        track_dict_car[track_id].append((xmin, ymin, xmax, ymax, frame_id))
    # print("Trackdict")
    # print(track_dict)
    print("Track number in car: {}".format(len(track_dict_car.keys())))
    #cv2_imshow(track_image_for_car) 

  if dets_moto.size != 0:
    trackers_moto = moto_tracker.update(dets_moto)
  # print('trackers')
  # print(trackers)

    for xmin, ymin, xmax, ymax, track_id in trackers_moto:
      track_id = int(track_id)
      #  Visualize tracker
      #cv2.rectangle(track_image_for_moto, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
      #cv2.putText(track_image_for_moto, "Moto Track " + str(track_id), (int(xmax), int(ymax)), cv2.FONT_HERSHEY_SIMPLEX, color=(0, 0, 255), fontScale=0.5)
      ##print(track_id)
      if track_id not in track_dict_moto.keys():
        track_dict_moto[track_id] = [(xmin, ymin, xmax, ymax, frame_id)]
      else:
        track_dict_moto[track_id].append((xmin, ymin, xmax, ymax, frame_id))
    # print("Trackdict")
    # print(track_dict)
    print("Track number in moto: {}".format(len(track_dict_moto.keys())))
    #cv2_imshow(track_image_for_moto) 


print("Finished")

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2000.jpg



 33%|███▎      | 2001/6010 [1:43:15<3:20:16,  3.00s/it]

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2001.jpg



 33%|███▎      | 2002/6010 [1:43:18<3:21:10,  3.01s/it]

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2002.jpg


...

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2003.jpg



 33%|███▎      | 2004/6010 [1:43:24<3:20:05,  3.00s/it]

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2004.jpg



 33%|███▎      | 2005/6010 [1:43:27<3:21:10,  3.01s/it]

/content/drive/My Drive/VietAI/VietAI-AdvancedClass-2020-10/Track/cam_01/2005.jpg


In [ ]:
for track_id, track_item in track_dict.items():
  print(track_id,track_item)

11083 [(123.69330883026123, 170.3914451599121, 700.0, 3.0, 700)]
11084 [(125.2143144607544, 168.5245704650879, 701.0, 3.0, 701)]
11085 [(129.0392518043518, 167.44558334350586, 702.0, 3.0, 702)]


In [ ]:
# Create an motobikes tracker with default parameter.
# Please read the sort documentation for the custom paramenters.

moto_tracker = Sort()

# If you want to track another vehicle class, you need to declare a new tracker.
# truck_tracker = Sort()

track_dict = {}

N_FRAMES = 20

for frame_id in range(1, N_FRAMES):
  image_path = os.path.join(image_dir, '{}.jpg'.format(frame_id))
  print("Image path :")
  print(image_path)
  image_np = load_image_into_numpy_array(image_path)


  im_width, im_height, _ = image_np.shape
  input_tensor = tf.convert_to_tensor(
      np.expand_dims(image_np, 0), dtype=tf.float32)
  detections, predictions_dict, shapes = detect_fn(input_tensor)

  # print("detection")
  # print(detections)
  # print("predictions_dict")
  # print(predictions_dict)

  boxes = detections['detection_boxes'][0]
  scores = detections['detection_scores'][0]
  classes = detections['detection_classes'][0]


  dets = []
  for bb, s, c in zip(boxes, scores, classes):
    ymin, xmin, ymax, xmax = bb.numpy()
    xmin, ymin, xmax, ymax = xmin*im_width, ymin*im_height, xmax*im_width, ymax*im_height
    if check_bbox_intersect_polygon(polygon, (xmin, ymin, xmax, ymax)):
      # check if the bbox is in ROI
      dets.append((frame_id, c.numpy(), xmin, ymin, xmax, ymax, s.numpy()))


  label_id_offset = 1
  image_np_with_detections = image_np.copy()


  dets = np.array(dets)
  # Only get the detections with the class label is '3' which indicate the motobike class.
  moto_dets = dets[dets[:,1]==3]
  moto_dets = np.array(moto_dets)

  trackers = moto_tracker.update(moto_dets)
  for xmin, ymin, xmax, ymax, track_id in trackers:
    track_id = int(track_id)
    # print(track_id)
    if track_id not in track_dict.keys():
      track_dict[track_id] = [(xmin, ymin, xmax, ymax, frame_id)]
    else:
      track_dict[track_id].append((xmin, ymin, xmax, ymax, frame_id))

In [ ]:
for track_id  in track_dict.keys():
  print("Track id {}".format(track_id))
  print("Track list : {}".format(track_dict[track_id]))

Track id 1
Track list : [(355.0, 478.0, 639.0, 720.0, 0), (387.12100445531587, 495.8763230112512, 650.8746008183557, 720.1218791461598, 1), (441.9052425526649, 491.8175808335184, 680.5609596047734, 696.0645636643371, 2), (470.1829410317819, 477.301992877689, 694.9855452026989, 672.1246979737714, 3), (496.2486679638231, 472.31270903923956, 706.0082650345993, 651.6452568898575, 4), (519.2568470099717, 460.192952807652, 718.366760352592, 629.9179471457672, 5), (543.7074907898475, 449.2703111239226, 731.8918935157353, 610.1920989628705, 6), (561.6501385029603, 440.2818575791472, 741.0883101154895, 593.117846329795, 7), (581.2563152734359, 431.19886981965436, 749.8239687415152, 577.0243830306038, 8), (599.1793985798193, 424.38718754032624, 758.1083172736429, 562.3296999884295, 9), (615.9230138110408, 417.08253780014155, 764.8907976896191, 548.0699133992875, 10), (639.4318991720571, 402.0355349667918, 777.3862468814849, 526.1159163784212, 11), (657.8543104958547, 391.45769957721, 784.3387011

In [ ]:
# moto_vector_list: list of tuples (first_point, last_point, last_frame_id)
# list of moto movement vector and the last frame_id when it is still in the ROI.

moto_vector_list = []
for tracker_id, tracker_list in track_dict.items():
  # print(track_id)
  if len(tracker_list) > 1:
    first = tracker_list[0]
    last = tracker_list[-1]
    first_point = ((first[2] - first[0])/2, (first[3] - first[1])/2)
    last_point = ((last[2] - last[0])/2, (last[3] - last[1])/2)
    moto_vector_list.append((first_point, last_point, last[4]))
    a = [first_point, last_point, last[4]]
    # print(a)
    
# print(moto_vector_list)  
# print(type(moto_vector_list))

In [ ]:
def cosin_similarity(a2d, b2d):
  
  a = np.array((a2d[1][0] - a2d[0][0], a2d[1][1 ]- a2d[0][1]))
  b = np.array((b2d[1][0] - b2d[0][1], b2d[1][1] - b2d[1][0]))
  return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))



In [ ]:
MOTO_CLASS_ID = 4

In [ ]:
 # Đếm số lương
 

Phát hiện MOI tương ứng với mỗi xe

In [ ]:
def counting_moi(paths, moto_vector_list):
  """
  Args:
    paths: List of MOI - (first_point, last_point)
    moto_vector_list: List of tuples (first_point, last_point, last_frame_id) 
  
  Returns:
    A list of tuples (frame_id, movement_id, vehicle_class_id)
  """
  moi_detection_list = []
  for moto_vector in moto_vector_list:
    max_cosin = -2
    movement_id = ''
    last_frame = 0
    for movement_label, movement_vector in paths.items():
      cosin = cosin_similarity(movement_vector, moto_vector)
      if cosin > max_cosin:
        max_cosin = cosin
        movement_id = movement_label
        last_frame = moto_vector[2]

    moi_detection_list.append((last_frame, movement_id, MOTO_CLASS_ID))
  return moi_detection_list


In [ ]:

moto_moi_detections = counting_moi(paths, moto_vector_list)

In [ ]:
print(moto_moi_detections)

[(21, '1', 4), (51, '1', 4), (51, '1', 4), (278, '1', 4), (395, '1', 4), (443, '1', 4), (769, '1', 4), (871, '1', 4), (896, '1', 4), (918, '1', 4), (1171, '1', 4), (1289, '1', 4), (1521, '1', 4), (1616, '1', 4), (1776, '1', 4), (2208, '1', 4), (2399, '1', 4), (2694, '1', 4), (2696, '1', 4), (2699, '1', 4), (2722, '1', 4), (3094, '1', 4), (3292, '1', 4), (3663, '1', 4), (3933, '1', 4), (4038, '1', 4), (4277, '1', 4), (4333, '1', 4), (4465, '1', 4), (5244, '1', 4), (5671, '1', 4), (5824, '1', 4), (5968, '1', 4)]


# Xuất kết quả theo định dạng nộp


In [ ]:
ls

README.md  result.txt  sample_data/  solution_baseline/


In [ ]:
%cd '/content/drive/My Drive/Result'

/content/drive/My Drive/Result


In [ ]:

result_filename = 'result_cam_06_class4_ver_2.txt'
video_id = 'cam_01'
with open(result_filename, 'w') as result_file:
  for frame_id, movement_id, vehicle_class_id in moto_moi_detections:
    result_file.write('{} {} {} {}\n'.format(video_id, frame_id  + 1, movement_id, vehicle_class_id))


In [ ]:
result_filename = 'result_cam_06_class4.txt'
video_id = 'cam_01'
with open(result_filename, 'w') as result_file:
  for frame_id, movement_id, vehicle_class_id in moto_moi_detections:
    result_file.write('{} {} {} {}\n'.format(video_id, frame_id , movement_id, vehicle_class_id))